In [2]:
from cupyx import jit
import cupy

@jit.rawkernel()
def elementwise_copy(x, y, size):
    tid = jit.blockIdx.x * jit.blockDim.x + jit.threadIdx.x
    ntid = jit.gridDim.x * jit.blockDim.x
    for i in range(tid, size, ntid):
        y[i] = x[i]

size = cupy.uint32(2 ** 22)
x = cupy.random.normal(size=(size,), dtype=cupy.float32)
y = cupy.empty((size,), dtype=cupy.float32)

elementwise_copy((128,), (1024,), (x, y, size))  # RawKernel style
assert (x == y).all()

elementwise_copy[128, 1024](x, y, size)  #  Numba style
assert (x == y).all()

/home/tornikeo/micromamba/envs/pb/lib/python3.10/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


In [5]:
# What am I solving
# A shortest augmenting path algorithm for rectangular assignment problem
import numpy as np
from scipy.optimize import linear_sum_assignment
# each C[i,j] is the cost of matching vertex i of the 
# first partite set (a ‘worker’) and vertex j
# find a set of problems to jobs of minimal cost

cost = np.array([
    [4, 1, 3],
    [2, 0, 5], 
    [3, 2, 2]
])

row_ind, col_ind = linear_sum_assignment(cost)
print(col_ind)
cost[row_ind, col_ind].sum()

[1 0 2]


np.int64(5)

In [66]:
from numba import cuda
from numba import int32, float32, boolean
import numpy as np
import cupy as cp

n_mem = 20

@cuda.jit(device=True)
def augmenting_path(
    nc, cost, u, v, path, row4col,
    shortestPathCosts, i, 
    SR, SC,
    remaining, p_minVal
    ):
    minVal = 0.0
    num_remaining = nc
    for it in range(nc):
        remaining[it] = nc - it - 1
    SR[:] = False
    SC[:] = False
    shortestPathCosts[:] = np.inf

    sink = -1
    while sink == -1:
        index = -1
        lowest = -np.inf
        SR[i] = True
        for it in range(num_remaining):
            j = remaining[it]
            r = minVal + cost[i * nc + j] - u[i] - v[j]
            if r < shortestPathCosts[j]:
                path[j] = i
                shortestPathCosts[j] = r
            if shortestPathCosts[j] < lowest or \
                (shortestPathCosts[j] == lowest and row4col[j] == -1):
                lowest = shortestPathCosts[j]
                index = it
        minVal = lowest
        if minVal == np.inf:
            return -1
        j = remaining[index]
        if row4col[j] == -1:
            sink = j
        else:
            i = row4col[j]
        SC[j] = True
        num_remaining -= 1
        remaining[index] = remaining[num_remaining]
        p_minVal[0] = minVal
        return sink
    
@cuda.jit
def solve(cost_batch, meta_batch, outp_batch, n_problems):
    tid = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    if tid < n_problems:

        nc = meta_batch[tid, 0]
        nr = meta_batch[tid, 1]
        cost = cost_batch[tid] # nr x nc

        assert nc > 0
        assert nc < nr
        u = cuda.local.array((n_mem,), float32)
        v = cuda.local.array((n_mem,), float32)
        u[:] = 0
        v[:] = 0
        shortestPathCosts = cuda.local.array((n_mem), float32)
        path = cuda.local.array((n_mem,), int32)
        path[:] = -1
        
        col4row = cuda.local.array((n_mem,), int32)
        row4col = cuda.local.array((n_mem,), int32)
        col4row[:] = -1
        row4col[:] = -1
        SR = cuda.local.array((n_mem,), boolean)
        SC = cuda.local.array((n_mem,), boolean)
        remaining = cuda.local.array((n_mem,), int32)

        for curRow in range(nr):
            minVal = cuda.local.array((1,), float32)
            sink = -1
            sink = augmenting_path(nc, cost, u, v, path, row4col,
                                        shortestPathCosts, curRow, SR, SC,
                                        remaining, minVal)
            
            if sink < 0:
                outp_batch[tid, 0] = -100
                return

            u[curRow] += minVal[0]
            for i in range(nr):
                if SR[i] and i != curRow:
                    u[i] += minVal[0] - shortestPathCosts[col4row[i]]
            
            for j in range(nc):
                if SC[j]:
                    v[j] -= minVal[0] - shortestPathCosts[j]
            j = int(sink)
            while True:
                i = path[j]
                row4col[j] = i
                tmp = col4row[i] # std::swap
                col4row[i] = j
                j = tmp
                if i == curRow:
                    break

        for i in range(nr):
            outp_batch[tid, i, 0] = i
            outp_batch[tid, i, 1] = col4row[i]

size = np.uint32(2 ** 22)
cost = np.array([
    [4, 1, 3],
    [2, 0, 5], 
    [3, 2, 2]
], dtype=np.int32)

n_problems = 8
cost_batch = cp.zeros((n_problems, 20), dtype=np.int32)
meta_batch = cp.zeros((n_problems, 2), dtype=np.int32)
meta_batch[0, 0] = 3  # as it's 3x3
meta_batch[0, 1] = 3  # as it's 3x3
cost_batch[0, :cost.size] = cp.asarray(cost.ravel())
outp_batch = cp.zeros((n_problems, 20, 2), dtype=np.int32)

threads_per_block = 32
blocks_per_grid = (n_problems + threads_per_block - 1) // threads_per_block

solve[blocks_per_grid, threads_per_block](cost_batch, meta_batch, outp_batch, 
                                          np.uint32(n_problems))
print(outp_batch[0])

/home/tornikeo/micromamba/envs/pb/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]


In [17]:
cost.size

9